In [ ]:
!git clone https://github.com/mlfoundations/rtfm.git
%cd rtfm

fatal: destination path 'rtfm' already exists and is not an empty directory.
/content/rtfm


In [ ]:
!pip install uv
!uv pip install -r requirements.txt --system

Using Python 3.10.12 environment at /usr
Audited 50 packages in 91ms


In [ ]:
!uv pip install git+https://github.com/jpgard/llama-recipes.git --system

Using Python 3.10.12 environment at /usr
Updating https://github.com/jpgard/llama-recipes.git (HEAD)
Updating https://github.com/jpgard/llama-recipes.git (HEAD)
Updating https://github.com/jpgard/llama-recipes.git (HEAD)
Updating https://github.com/jpgard/llama-recipes.git (HEAD)
Updating https://github.com/jpgard/llama-recipes.git (HEAD)
Updating https://github.com/jpgard/llama-recipes.git (HEAD)
 Updated https://github.com/jpgard/llama-recipes.git (186213f)
Resolved 122 packages in 1.18s
Audited 122 packages in 0.24ms


In [ ]:
!uv pip install -e . --system

Using Python 3.10.12 environment at /usr
Audited 1 package in 68ms


In [ ]:
!uv pip install --no-deps git+https://github.com/mlfoundations/tableshift.git --system

Using Python 3.10.12 environment at /usr
Updating https://github.com/mlfoundations/tableshift.git (HEAD)
Updating https://github.com/mlfoundations/tableshift.git (HEAD)
Updating https://github.com/mlfoundations/tableshift.git (HEAD)
Updating https://github.com/mlfoundations/tableshift.git (HEAD)
Updating https://github.com/mlfoundations/tableshift.git (HEAD)
Updating https://github.com/mlfoundations/tableshift.git (HEAD)
 Updated https://github.com/mlfoundations/tableshift.git (fca9429)
Resolved 1 package in 882ms
Audited 1 package in 0.24ms


In [ ]:
!uv pip install https://github.com/mlfoundations/tabliblib.git --system

Using Python 3.10.12 environment at /usr
Updating https://github.com/mlfoundations/tabliblib.git (HEAD)
Updating https://github.com/mlfoundations/tabliblib.git (HEAD)
Updating https://github.com/mlfoundations/tabliblib.git (HEAD)
Updating https://github.com/mlfoundations/tabliblib.git (HEAD)
Updating https://github.com/mlfoundations/tabliblib.git (HEAD)
Updating https://github.com/mlfoundations/tabliblib.git (HEAD)
 Updated https://github.com/mlfoundations/tabliblib.git (95f5ae4)
Resolved 1 package in 865ms
Audited 1 package in 0.20ms


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, LlamaForCausalLM, AutoConfig

from rtfm.configs import SerializerConfig, TrainConfig, TokenizerConfig
from rtfm.inference_utils import InferenceModel
from rtfm.serialization.serializers import get_serializer
from rtfm.tokenization.text import prepare_tokenizer

train_config = TrainConfig(model_name="mlfoundations/tabula-8b", context_length=8192)

tokenizer_config = TokenizerConfig()

# Load the configuration
config = AutoConfig.from_pretrained(train_config.model_name)

# Set the torch_dtype to bfloat16 which matches TabuLa train/eval setup
config.torch_dtype = 'bfloat16'

device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)

model = LlamaForCausalLM.from_pretrained(
    train_config.model_name, device_map="cuda", config=config).to(device)

tokenizer = AutoTokenizer.from_pretrained(train_config.model_name)
serializer = get_serializer(SerializerConfig())

tokenizer, model = prepare_tokenizer(
    model,
    tokenizer=tokenizer,
    pretrained_model_name_or_path=train_config.model_name,
    model_max_length=train_config.context_length,
    use_fast_tokenizer=tokenizer_config.use_fast_tokenizer,
    serializer_tokens_embed_fn=tokenizer_config.serializer_tokens_embed_fn,
    serializer_tokens=serializer.special_tokens
    if tokenizer_config.add_serializer_tokens
    else None,
)

inference_model = InferenceModel(model=model, tokenizer=tokenizer, serializer=serializer)

read user yaml files: 0it [00:00, ?it/s]
/usr/local/lib/python3.10/dist-packages/llama_recipes/model_checkpointing/checkpoint_handler.py:18: DeprecationWarning: `torch.distributed._shard.checkpoint` will be deprecated, use `torch.distributed.checkpoint` instead
  from torch.distributed._shard.checkpoint import (
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Pleas

cuda


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def is_datetime_or_timedelta_dtype(ser):
    return False
pd.core.dtypes.common.is_datetime_or_timedelta_dtype = is_datetime_or_timedelta_dtype

labeled_examples = pd.DataFrame(
    [
        {"location": "New York", "temperature": 22, "humidity": 65, "wind_speed": 12, "pressure": 1012, "month": "July",
         "weather_yesterday": "Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Partly Sunny"},
        {"location": "Los Angeles", "temperature": 26, "humidity": 60, "wind_speed": 7, "pressure": 1015,
         "month": "July", "weather_yesterday": "Partly Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Sunny"},
        {"location": "Chicago", "temperature": 18, "humidity": 70, "wind_speed": 15, "pressure": 1008, "month": "July",
         "weather_yesterday": "Partly Cloudy", "precipitation": 0.1, "visibility": 8, "weather_today": "Cloudy"},
        {"location": "Houston", "temperature": 30, "humidity": 80, "wind_speed": 10, "pressure": 1010, "month": "July",
         "weather_yesterday": "Rain", "precipitation": 0.5, "visibility": 7, "weather_today": "Rain"},
        {"location": "Phoenix", "temperature": 35, "humidity": 20, "wind_speed": 5, "pressure": 1005, "month": "July",
         "weather_yesterday": "Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Sunny"},
        {"location": "Philadelphia", "temperature": 24, "humidity": 75, "wind_speed": 14, "pressure": 1009,
         "month": "July", "weather_yesterday": "Partly Cloudy", "precipitation": 0.2, "visibility": 9,
         "weather_today": "Partly Cloudy"},
        {"location": "San Antonio", "temperature": 28, "humidity": 68, "wind_speed": 11, "pressure": 1011,
         "month": "July", "weather_yesterday": "Rain", "precipitation": 0.4, "visibility": 8, "weather_today": "Rain"},
        {"location": "San Diego", "temperature": 22, "humidity": 65, "wind_speed": 10, "pressure": 1014,
         "month": "July", "weather_yesterday": "Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Partly Sunny"},
        {"location": "Dallas", "temperature": 27, "humidity": 72, "wind_speed": 9, "pressure": 1007, "month": "July",
         "weather_yesterday": "Partly Cloudy", "precipitation": 0.3, "visibility": 9, "weather_today": "Cloudy"},
    ]
)
target_example = pd.DataFrame(
    [
        {"location": "San Jose", "temperature": 23, "humidity": 55, "wind_speed": 8, "pressure": 1013, "month": "July",
         "weather_yesterday": "Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Sunny"},
    ]
)

inference_model = InferenceModel(model=model, tokenizer=tokenizer, serializer=serializer)

output = inference_model.predict(
    target_example=target_example,
    target_colname="weather_today",
    target_choices=["Sunny", "Partly Sunny", "Cloudy", "Partly Cloudy", "Rain"],
    labeled_examples=labeled_examples,
)
print(f"Prediction for sample \n {target_example} \n is: {output}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prediction for sample 
    location  temperature  humidity  wind_speed  pressure month  \
0  San Jose           23        55           8      1013  July   

  weather_yesterday  precipitation  visibility weather_today  
0             Sunny              0          10         Sunny   
 is: Sunny


In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

target_example = pd.DataFrame(
    [
        {"location": "San Jose", "temperature": 27, "humidity": 45, "wind_speed": 3, "pressure": 1007, "month": "July",
         "weather_yesterday": "Partly Cloudy", "precipitation": 0, "visibility": 7}, #, "weather_today": "Sunny"},
    ]
)

inference_model = InferenceModel(model=model, tokenizer=tokenizer, serializer=serializer)

output = inference_model.predict(
    target_example=target_example,
    target_colname="weather_today",
    target_choices=["Sunny", "Partly Sunny", "Cloudy", "Partly Cloudy", "Rain"],
    # labeled_examples=labeled_examples,
)
print(f"Prediction for sample \n {target_example} \n is: {output}")

       'month', 'weather_yesterday', 'precipitation', 'visibility'],
      dtype='object'). Adding a dummy placeholder with empty values for preprocessing. This behavior is expected if your target samples do not contain the target column at all.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prediction for sample 
    location  temperature  humidity  wind_speed  pressure month  \
0  San Jose           27        45           3      1007  July   

  weather_yesterday  precipitation  visibility  weather_today  
0     Partly Cloudy              0           7            NaN   
 is: Partly Sunny


In [ ]:
from rtfm.serialization.serialization_utils import discretize_continuous_column

examples = pd.DataFrame(
    [
    {"location": "New York", "size_sqft": 1200, "bedrooms": 3, "bathrooms": 2, "age": 10, "lot_size_acres": 0.15, "garage": True, "price": 850},
    {"location": "Los Angeles", "size_sqft": 1500, "bedrooms": 4, "bathrooms": 3, "age": 8, "lot_size_acres": 0.25, "garage": True, "price": 950},
    {"location": "Chicago", "size_sqft": 1300, "bedrooms": 3, "bathrooms": 2, "age": 15, "lot_size_acres": 0.2, "garage": False, "price": 700},
    {"location": "Houston", "size_sqft": 1700, "bedrooms": 4, "bathrooms": 3, "age": 5, "lot_size_acres": 0.3, "garage": True, "price": 650},
    {"location": "Phoenix", "size_sqft": 1600, "bedrooms": 3, "bathrooms": 2, "age": 7, "lot_size_acres": 0.25, "garage": True, "price": 750},
    {"location": "Philadelphia", "size_sqft": 1400, "bedrooms": 3, "bathrooms": 2, "age": 12, "lot_size_acres": 0.18, "garage": False, "price": 600},
    {"location": "San Antonio", "size_sqft": 1800, "bedrooms": 4, "bathrooms": 3, "age": 3, "lot_size_acres": 0.4, "garage": True, "price": 700},
    {"location": "San Diego", "size_sqft": 1550, "bedrooms": 3, "bathrooms": 2, "age": 9, "lot_size_acres": 0.22, "garage": True, "price": 850},
    {"location": "Dallas", "size_sqft": 1450, "bedrooms": 3, "bathrooms": 2, "age": 11, "lot_size_acres": 0.19, "garage": True, "price": 700},
    {"location": "San Jose", "size_sqft": 1600, "bedrooms": 4, "bathrooms": 3, "age": 6, "lot_size_acres": 0.2, "garage": False, "price": 800},
    {"location": "Seattle", "size_sqft": 1800, "bedrooms": 4, "bathrooms": 2, "age": 10, "lot_size_acres": 0.2, "garage": False, "price": 925},
]
)

examples["price"] = discretize_continuous_column(examples["price"], num_buckets=4)
target_choices = examples["price"].unique().tolist()

target_example = examples.iloc[[0]]
labeled_examples = examples.iloc[1:]

inference_model = InferenceModel(model=model, tokenizer=tokenizer, serializer=serializer)

output = inference_model.predict(
    target_example=target_example,
    target_colname="price",
    target_choices=target_choices,
    labeled_examples=labeled_examples,
)
print(f"Prediction for sample \n {target_example} \n is: {output}")

/content/rtfm/rtfm/datasets/data_utils.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=columns_to_drop, inplace=True)
/content/rtfm/rtfm/datasets/data_utils.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=columns_to_drop, inplace=True)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prediction for sample 
    location  size_sqft  bedrooms  bathrooms  age  lot_size_acres  garage  \
0  New York       1200         3          2   10            0.15    True   

                price  
0  greater than 850.0   
 is: between 750.0 and 850.0


In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

target_example = pd.DataFrame(
    [
    {"location": "New York", "size_sqft": 1200, "bedrooms": 3, "bathrooms": 2, "age": 10, "lot_size_acres": 0.15, "garage": True}, #, "price": 850},
]
)

inference_model = InferenceModel(model=model, tokenizer=tokenizer, serializer=serializer)

output = inference_model.predict(
    target_example=target_example,
    target_colname="price",
    target_choices=target_choices,
)
print(f"Prediction for sample \n {target_example} \n is: {output}")

       'lot_size_acres', 'garage'],
      dtype='object'). Adding a dummy placeholder with empty values for preprocessing. This behavior is expected if your target samples do not contain the target column at all.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prediction for sample 
    location  size_sqft  bedrooms  bathrooms  age  lot_size_acres  garage  \
0  New York       1200         3          2   10            0.15    True   

   price  
0    NaN   
 is: greater than 850.0
